In [1]:
import datalabframework as dlf

In [2]:
dlf.version_info

(0, 5, 10)

In [3]:
engine = dlf.engines.get('spark')
spark = engine.context()

PYSPARK_SUBMIT_ARGS:  --packages mysql:mysql-connector-java:8.0.12 pyspark-shell


In [4]:
md = dlf.params.metadata()
for resource in md['resources']:
    ds = md['resources'][resource]
    if ds['provider']=='source':
        print(ds['path'])
        # read
        engine.read(resource).show(5)

actor
2018-10-18 05:15:50,887 - INFO - ffc0af7 - datalabframework-demos.git - jovyan - ingest.ipynb - engine.read - {'format': 'rdbms', 'service': 'mysql', 'path': 'actor', 'url': 'jdbc:mysql://mysql:3306/sakila'}
+--------+----------+---------+-------------------+
|actor_id|first_name|last_name|        last_update|
+--------+----------+---------+-------------------+
|     105|    SIDNEY|    CROWE|2006-02-15 04:34:33|
|     172|   GROUCHO| WILLIAMS|2006-02-15 04:34:33|
|      74|     MILLA|   KEITEL|2006-02-15 04:34:33|
|      48|   FRANCES|DAY-LEWIS|2006-02-15 04:34:33|
|      65|    ANGELA|   HUDSON|2006-02-15 04:34:33|
+--------+----------+---------+-------------------+
only showing top 5 rows

address
2018-10-18 05:15:53,018 - INFO - ffc0af7 - datalabframework-demos.git - jovyan - ingest.ipynb - engine.read - {'format': 'rdbms', 'service': 'mysql', 'path': 'address', 'url': 'jdbc:mysql://mysql:3306/sakila'}
+----------+--------------------+--------+---------+-------+-----------+---

In [9]:
day = md = dlf.params.metadata()
for resource in md['resources']:
    ds = md['resources'][resource]
    if ds['provider']=='source':
        print(ds['path'])
        # read
        df_src = engine.read(resource)
        
        # define target path
        target_path = '{}.{}'.format(ds['provider'],ds['path'])
        print(target_path)
        
        # write
        engine.write(df_src, path=target_path, provider='target', mode='overwrite')

actor
2018-10-18 05:18:00,782 - INFO - ffc0af7 - datalabframework-demos.git - jovyan - ingest.ipynb - engine.read - {'format': 'rdbms', 'service': 'mysql', 'path': 'actor', 'url': 'jdbc:mysql://mysql:3306/sakila'}
source.actor
2018-10-18 05:18:02,981 - INFO - ffc0af7 - datalabframework-demos.git - jovyan - ingest.ipynb - engine.write - {'format': 'parquet', 'service': 'hdfs', 'path': 'source.actor', 'url': 'hdfs://hdfs-nn:8020/data/ingest/source.actor'}
address
2018-10-18 05:18:03,175 - INFO - ffc0af7 - datalabframework-demos.git - jovyan - ingest.ipynb - engine.read - {'format': 'rdbms', 'service': 'mysql', 'path': 'address', 'url': 'jdbc:mysql://mysql:3306/sakila'}
source.address
2018-10-18 05:18:04,476 - INFO - ffc0af7 - datalabframework-demos.git - jovyan - ingest.ipynb - engine.write - {'format': 'parquet', 'service': 'hdfs', 'path': 'source.address', 'url': 'hdfs://hdfs-nn:8020/data/ingest/source.address'}
category
2018-10-18 05:18:04,661 - INFO - ffc0af7 - datalabframework-demos

### Ingest from source to target 

In [1]:
import datalabframework as dlf

In [2]:
dlf.data.metadata('resources.actor')
engine = dlf.engines.get('spark')
spark = engine.context()

PYSPARK_SUBMIT_ARGS:  --packages mysql:mysql-connector-java:8.0.12 pyspark-shell


In [3]:
# df =  engine.read('resources.actor')
# df.groupby('last_update').count().show()

In [7]:
engine.ingest(src_resource='resources.language', dest_path='ingest.language', dest_provider='target')

condition: last_update < "2018-10-18T00:00:00" and last_update >= "2006-02-15T00:00:00" 
ingest.language/schema target
2018-10-18 07:02:17,762 - WARNING - ffc0af7 - datalabframework-demos.git - jovyan - ingest.ipynb - message - No schema file for ingest.language, target
schema date 2018-10-18 00:00:00
2018-10-18 07:02:17,998 - INFO - ffc0af7 - datalabframework-demos.git - jovyan - ingest.ipynb - engine.read - {'format': 'rdbms', 'service': 'mysql', 'path': 'language', 'url': 'jdbc:mysql://mysql:3306/sakila'}
{'provider': {'format': 'parquet', 'hostname': 'hdfs-nn', 'path': 'data/ingest', 'service': 'hdfs', 'write': {'coalesce': 2, 'options': {'mode': 'append'}}}, 'resource': {'path': 'ingest.language', 'provider': 'target'}, 'url': 'hdfs://hdfs-nn:8020/data/ingest/ingest.language'}
ingest.language/2018-10-18
2018-10-18 07:02:18,247 - WARNING - ffc0af7 - datalabframework-demos.git - jovyan - ingest.ipynb - message - Nothing to read from destination: ingest.language/2018-10-18, target
20

In [8]:
import pandas as pd
from datetime import datetime
import json

df = pd.DataFrame(columns=['date', 'schema'], data=[[datetime.today(), json.dumps({'ohlala':42})]])
spark = engine.context()
spark.createDataFrame(df)

DataFrame[date: timestamp, schema: string]

In [9]:
engine.read(path='ingest.language/schema', provider='target').show(5)

2018-10-18 07:02:20,173 - INFO - ffc0af7 - datalabframework-demos.git - jovyan - ingest.ipynb - engine.read - {'format': 'parquet', 'service': 'hdfs', 'path': 'ingest.language/schema', 'url': 'hdfs://hdfs-nn:8020/data/ingest/ingest.language/schema'}
+--------------------+--------------------+
|                date|              schema|
+--------------------+--------------------+
|2018-10-18 07:02:...|{"fields":[{"meta...|
+--------------------+--------------------+



## pre checks

schema checks:

 - get table schema from source
 - compare with reference schema
 
value checks:
 - null
 - invalid 

## Ingest strategy

#### **what to read?**
 
FULL scan:  
tables are lost or never ingested before
 
  - cold start
  - disaster recovery from source
     
INCREMENTAL scan:  
We need a way to select/filter new data

  - time based (default, i.e. last_update column)
  - index based (if index is auto incrementing)
  - hash compare indexes (expensive)
  - full rescan and tag with ingest date
 
#### **where to write?**

  - define a naming convention for the target tables (default schema version: `latest`)  
    `<source>/<db-name>/<table-name>/<schema-version-date>`
    
#### **exceptions**
: what to do? (show error)

### HDFS

In [5]:
# read back from hdfs in parquet format
df_trg = engine.read('target', 'ingest')
df_trg.where(col(colname) => 'datetime')
df_trg.show()

hdfs://hdfs-nn:8020//data/ingest/extract/sakila.actor
+--------+----------+------------+-------------------+
|actor_id|first_name|   last_name|        last_update|
+--------+----------+------------+-------------------+
|       1|  PENELOPE|     GUINESS|2006-02-15 04:34:33|
|       2|      NICK|    WAHLBERG|2006-02-15 04:34:33|
|       3|        ED|       CHASE|2006-02-15 04:34:33|
|       4|  JENNIFER|       DAVIS|2006-02-15 04:34:33|
|       5|    JOHNNY|LOLLOBRIGIDA|2006-02-15 04:34:33|
|       6|     BETTE|   NICHOLSON|2006-02-15 04:34:33|
|       7|     GRACE|      MOSTEL|2006-02-15 04:34:33|
|       8|   MATTHEW|   JOHANSSON|2006-02-15 04:34:33|
|       9|       JOE|       SWANK|2006-02-15 04:34:33|
|      10| CHRISTIAN|       GABLE|2006-02-15 04:34:33|
|      11|      ZERO|        CAGE|2006-02-15 04:34:33|
|      12|      KARL|       BERRY|2006-02-15 04:34:33|
|      13|       UMA|        WOOD|2006-02-15 04:34:33|
|      14|    VIVIEN|      BERGEN|2006-02-15 04:34:33|
|      15| 

## post checks

In [6]:
assert(df_src.subtract(df_trg).count()==0)

In [ ]:
engine.read(path='abcd', provider='dsds')
write(df, path='abcd', provider='ingest')

engine.read('resource_alias')
engine.write(df, 'resource_alias')
